Chapter 28
More Aggregations
.transform method and .filter method



28.1 Aggregations while keeping rows.

In [1]:
#clears the output variables

%reset -f

In [10]:
import pandas as pd

import collections

import catboost as cb

import numpy as np

In [3]:
def get_uniq_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    uniq_cols = []
    for cols in counter.values():
        if len(cols) == 1:
            uniq_cols.extend(cols)
    return uniq_cols

In [4]:
#books code
def prep_for_ml(df):
    # remove pandas types
    return (df
     .assign(**{col:df[col].astype(float) 
               for col in df.select_dtypes('number')},
             **{col:df[col].astype(str).fillna('') 
               for col in df.select_dtypes(['object', 'category'])})
    )

In [5]:
def predict_col(df, col):
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i,typ in enumerate(df.drop(columns=[col]).dtypes):
        if str(typ) == 'object':
            cat_idx.append(i)
    X = (missing
         .drop(columns=[col])
         .values
        )
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)


In [11]:
# books code
def tweak_jb(jb):
     uniq_cols = get_uniq_cols(jb)
     return (jb
         [uniq_cols]
         .rename(columns=lambda c: c.replace('.', '_'))
         .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
                    .astype('Int64'),
                are_you_datascientist=lambda df_:df_
                    .are_you_datascientist
                    .replace({'Yes': True, 'No': False, np.nan: False}),
                company_size=lambda df_:df_.company_size.replace({
                    'Just me': 1, 'Not sure': np.nan,
                    'More than 5,000': 5000, '2–10': 2, '11–50':11,
                    '51–500': 51, '501–1,000':501,
                    '1,001–5,000':1001}).astype('Int64'),
                country_live=lambda df_:df_.country_live
                    .astype('category'),
                employment_status=lambda df_:df_.employment_status
                     .fillna('Other').astype('category'),
                is_python_main=lambda df_:df_.is_python_main
                     .astype('category'),
                team_size=lambda df_:df_.team_size
                    .str.split(r'-', n=1, expand=True)
                    .iloc[:,0].replace('More than 40 people', 41)
                    .where(df_.company_size!=1, 1).astype(float),
                years_of_coding=lambda df_:df_.years_of_coding
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python_years=lambda df_:df_.python_years
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python3_ver=lambda df_:df_.python3_version_most
                     .str.replace('_', '.').str.extract(r'(\d\.\d)')
                     .astype(float),
                use_python_most=lambda df_:df_.use_python_most
                     .fillna('Unknown')
               )
        .assign(team_size=lambda df_:predict_col(df_, 'team_size')
             .astype(int))
        .drop(columns=['python2_version_most'])
        .dropna()
        #.astype({'age':'int64','company_size':'int64'})   # would throw a 'Int64' error if not converted to 'int64'
    )    
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
   '2020-jetbrains-python-survey.csv'
jb = pd.read_csv(url)
jb2 = tweak_jb(jb)


C:\Users\wgrom\AppData\Local\Temp\ipykernel_7960\191578454.py:47: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


Learning rate set to 0.5
0:	learn: 2.9695218	total: 186ms	remaining: 3.54s
1:	learn: 2.8766539	total: 231ms	remaining: 2.08s
2:	learn: 2.8387189	total: 277ms	remaining: 1.57s
3:	learn: 2.8028751	total: 327ms	remaining: 1.31s
4:	learn: 2.7899957	total: 376ms	remaining: 1.13s
5:	learn: 2.7749439	total: 420ms	remaining: 979ms
6:	learn: 2.7719128	total: 464ms	remaining: 862ms
7:	learn: 2.7649792	total: 512ms	remaining: 767ms
8:	learn: 2.7649588	total: 548ms	remaining: 670ms
9:	learn: 2.7630617	total: 590ms	remaining: 590ms
10:	learn: 2.7625779	total: 629ms	remaining: 515ms
11:	learn: 2.7515902	total: 667ms	remaining: 445ms
12:	learn: 2.7513459	total: 704ms	remaining: 379ms
13:	learn: 2.7445634	total: 741ms	remaining: 317ms
14:	learn: 2.7443257	total: 775ms	remaining: 258ms
15:	learn: 2.7423142	total: 809ms	remaining: 202ms
16:	learn: 2.7419143	total: 845ms	remaining: 149ms
17:	learn: 2.7399387	total: 885ms	remaining: 98.3ms
18:	learn: 2.7384296	total: 921ms	remaining: 48.5ms
19:	learn: 2.7